# Test of CSV file produced using ROOT converter

In this notebook I'll test the macro, written in python, used for converting a ROOT structure (Tree) into a CSV file for ML analysis.

The CSV is located inside the directory output, created by the python macro **examplemacro.py**:

In [ ]:
!ls /Users/tommaso/TESI_MAGISTRALE/MuonPOGAnalysisTemplate/

In [1]:
%cd /Users/tommaso/TESI_MAGISTRALE/MuonPOGAnalysisTemplate/output
!head -5 output_bxcut.csv

/Users/tommaso/TESI_MAGISTRALE/MuonPOGAnalysisTemplate/output







To analyze it, we need the ROOT python module and pandas for the CSV analysis:

In [2]:
from ROOT import TCanvas,TH1F
import pandas as pd
import numpy as np

Welcome to JupyROOT 6.12/04


Now we need to read the CSV:

In [3]:
df2 = pd.read_csv('./output_bxcut.csv')

This creates a pandas db that can be displayed showing its content:

In [ ]:
df2

In [4]:
a = 1
new_column = []
row_list=[]
final_row=[]
for column in df2.columns.values.tolist():
    if column==df2.columns.values.tolist()[0]:
        new_column.append(column)
        continue
    for count in range(1,df2["Event"].value_counts().max()+1):
        new_column.append(column + str(count))

df = pd.DataFrame(columns=new_column)
for index, row in df2.iterrows():
    if (row["Event"] == a):
        row_list.append(row.tolist())
        continue
    elif(row["Event"] != a):
        for i in range(0,len(df2.columns)):
            if i==0:
                final_row.append(row_list[0][0])
                continue
            for j in range(0,len(row_list)):
                final_row.append(row_list[j][i])
                if j == len(row_list)-1:   
                    if len(row_list) != df2["Event"].value_counts().max():
                        final_row += [np.nan]*(df2["Event"].value_counts().max()-len(row_list))
        a = row["Event"]
        df.loc[row["Event"]-1] = final_row
        del row_list[:]
        del final_row[:]
        row_list.append(row.tolist())

for i in range(0,len(df2.columns)):
    if i==0:
        final_row.append(row_list[0][0])
        continue
    for j in range(0,len(row_list)):
        final_row.append(row_list[j][i])
        if j == len(row_list)-1:   
            if len(row_list) != df2["Event"].value_counts().max():
                final_row += [np.nan]*(df2["Event"].value_counts().max()-len(row_list))
df.loc[row["Event"]] = final_row

['Event', 'dtPrimitive.id_r1', 'dtPrimitive.id_r2', 'dtPrimitive.id_r3', 'dtPrimitive.id_r4', 'dtPrimitive.id_r5', 'dtPrimitive.id_eta1', 'dtPrimitive.id_eta2', 'dtPrimitive.id_eta3', 'dtPrimitive.id_eta4', 'dtPrimitive.id_eta5', 'dtPrimitive.id_phi1', 'dtPrimitive.id_phi2', 'dtPrimitive.id_phi3', 'dtPrimitive.id_phi4', 'dtPrimitive.id_phi5', 'dtPrimitive.bxTrackFinder()1', 'dtPrimitive.bxTrackFinder()2', 'dtPrimitive.bxTrackFinder()3', 'dtPrimitive.bxTrackFinder()4', 'dtPrimitive.bxTrackFinder()5', 'dtPrimitive.phiGlb()1', 'dtPrimitive.phiGlb()2', 'dtPrimitive.phiGlb()3', 'dtPrimitive.phiGlb()4', 'dtPrimitive.phiGlb()5', 'genParticle.pt1', 'genParticle.pt2', 'genParticle.pt3', 'genParticle.pt4', 'genParticle.pt5', 'dtPrimitive.bx1', 'dtPrimitive.bx2', 'dtPrimitive.bx3', 'dtPrimitive.bx4', 'dtPrimitive.bx5']


In [5]:
df

,Event,dtPrimitive.id_r1,dtPrimitive.id_r2,dtPrimitive.id_r3,dtPrimitive.id_r4,dtPrimitive.id_r5,dtPrimitive.id_eta1,dtPrimitive.id_eta2,dtPrimitive.id_eta3,dtPrimitive.id_eta4,...,genParticle.pt1,genParticle.pt2,genParticle.pt3,genParticle.pt4,genParticle.pt5,dtPrimitive.bx1,dtPrimitive.bx2,dtPrimitive.bx3,dtPrimitive.bx4,dtPrimitive.bx5
1.0,1.0,1.0,2.0,3.0,4.0,NaN,1.0,1.0,2.0,2.0,...,9.452828,9.452828,9.452828,9.452828,NaN,0.0,0.0,0.0,0.0,NaN
2.0,2.0,4.0,3.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,...,164.478928,164.478928,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
3.0,3.0,1.0,3.0,4.0,NaN,NaN,0.0,0.0,0.0,NaN,...,91.713860,91.713860,91.713860,NaN,NaN,0.0,0.0,0.0,NaN,NaN
4.0,4.0,2.0,3.0,4.0,1.0,NaN,-2.0,-2.0,-2.0,-1.0,...,139.442352,139.442352,139.442352,139.442352,NaN,0.0,0.0,0.0,0.0,NaN
5.0,5.0,1.0,3.0,4.0,NaN,NaN,1.0,2.0,2.0,NaN,...,198.223480,198.223480,198.223480,NaN,NaN,0.0,0.0,0.0,NaN,NaN
6.0,6.0,3.0,NaN,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,...,47.726688,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
7.0,7.0,4.0,2.0,3.0,NaN,NaN,1.0,1.0,1.0,NaN,...,146.894150,146.894150,146.894150,NaN,NaN,0.0,0.0,0.0,NaN,NaN
8.0,8.0,1.0,2.0,3.0,4.0,NaN,1.0,2.0,2.0,2.0,...,149.884445,149.884445,149.884445,149.884445,NaN,0.0,0.0,0.0,0.0,NaN
9.0,9.0,2.0,3.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,...,74.841820,74.841820,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
10.0,10.0,1.0,2.0,3.0,4.0,NaN,0.0,0.0,0.0,0.0,...,159.808716,159.808716,159.808716,159.808716,NaN,0.0,0.0,0.0,0.0,NaN
